## Thuật toán tính giá trị P(x) tại x = x' với mọi bậc đạo hàm  

In [2]:
import numpy as np
import pandas as pd
import math

# (Hàm định dạng này sẽ hữu ích ở ô sau)
def format_polynomial_descending(coeffs_desc):
    """Hàm định dạng đa thức, sắp xếp theo bậc giảm dần."""
    poly_str = ""; n = len(coeffs_desc) - 1
    for i, coeff in enumerate(coeffs_desc):
        if np.isclose(coeff, 0): continue
        power = n - i; sign = " - " if coeff < 0 else " + "; coeff_abs = abs(coeff)
        if poly_str == "": sign = "" if coeff > 0 else "-"
        if np.isclose(coeff_abs, 1) and power != 0: coeff_str = ""
        else: coeff_str = f"{coeff_abs:g}"
        if power == 1: var_str = "x"
        elif power == 0: var_str = ""
        else: var_str = f"x^{power}"
        separator = "*" if coeff_str != "" and var_str != "" else ""
        poly_str += f" {sign} {coeff_str}{separator}{var_str}"
    return poly_str.strip() if poly_str else "0"

def load_coeffs_from_file(filename):
    """Tải hệ số đa thức từ file txt."""
    try:
        # ndmin=1 để đảm bảo nó là array ngay cả khi chỉ có 1 hệ số
        coeffs = np.loadtxt(filename, ndmin=1).astype(float)
        # Xử lý trường hợp file chỉ có 1 số
        if coeffs.ndim == 0:
             coeffs = np.array([coeffs])
        return coeffs
    except FileNotFoundError:
        raise FileNotFoundError(f"LỖI: Không tìm thấy file '{filename}'. Vui lòng tạo file này.")
    except ValueError:
        raise ValueError(f"LỖI: File '{filename}' chứa dữ liệu không phải là số.")

# --- Chương trình chính cho Ô 1 ---
file_coeffs = 'coeffs.txt'
try:
    # Tải hệ số. Giả định file lưu theo bậc giảm dần (a_n, a_{n-1}, ..., a_0)
    p_coeffs_descending = load_coeffs_from_file(file_coeffs)
    print(f"Đã tải thành công các hệ số đa thức (bậc giảm dần):")
    print(p_coeffs_descending)
    
    # (Tùy chọn) In ra đa thức để xác nhận
    print(f"\nĐa thức P(x) = {format_polynomial_descending(p_coeffs_descending)}")
    
except (FileNotFoundError, ValueError) as e:
    print(e)

Đã tải thành công các hệ số đa thức (bậc giảm dần):
[23. 34.  1. 56.]

Đa thức P(x) = 23*x^3  +  34*x^2  +  x  +  56


In [ ]:
# Giả định 'p_coeffs_descending' đã được định nghĩa ở ô trên.

# --- 1. Định nghĩa giá trị x' cần tính ---
x_prime = 1.5                         # Bạn có thể thay đổi giá trị x' (điểm tính) ở đây

# --- 2. Định nghĩa hàm tính đạo hàm bằng Horner lặp ---
def tinh_P_va_cac_dao_ham(coeffs_desc, x_val):
    """
    Tính giá trị của P(x') và tất cả các đạo hàm của nó P'(x'), P''(x'),...
    bằng phương pháp Horner lặp.
    """
    
    print("\n" + "="*50)
    print(f"Sử dụng các hệ số P(x) (bậc giảm dần): {coeffs_desc}")
    print(f"Tính các đạo hàm tại x' = {x_val}")
    
    current_coeffs = list(coeffs_desc)
    n = len(current_coeffs) - 1
    summary_data = []
    k_factorial = 1.0

    for k in range(n + 1):
        print("\n" + "="*50 + f"\n--- BƯỚC {k+1}: TÍNH TOÁN CHO ĐẠO HÀM CẤP {k} ---")
        
        m = len(current_coeffs)
        deg = m - 1
        
        header = [f'a_{deg-i}' for i in range(deg + 1)]
        row_coeffs = [round(c, 10) for c in current_coeffs]
        row_products = ['']
        row_results = []

        b = current_coeffs[0]
        row_results.append(round(b, 10))
        
        quotient_coeffs = [b]
        
        for i in range(1, m):
            product = b * x_val
            row_products.append(round(product, 10))
            b = product + current_coeffs[i]
            row_results.append(round(b, 10))
            if i < m - 1:
                quotient_coeffs.append(b)
        
        remainder = b
        
        df = pd.DataFrame(
            [row_coeffs, row_products, row_results],
            index=['Hệ số vào', f'Nhân với x={x_val}', 'Kết quả'],
            columns=header
        )
        display(df)
        
        if k > 0:
            k_factorial *= k
        
        derivative_value = remainder * k_factorial
        summary_data.append([f"P^({k})(x')", derivative_value])
        
        print(f"-> Số dư R_{k+1} = {remainder:g}")
        print(f"-> Giá trị đạo hàm P^({k})(x') = R_{k+1} * {k}! = {remainder:g} * {k_factorial} = {derivative_value:g}")

        current_coeffs = quotient_coeffs

    # --- In bảng tổng kết cuối cùng ---
    column_name_value = f"Giá trị tại x = {x_val}"
    summary_df = pd.DataFrame(summary_data, columns=["Đạo hàm", column_name_value])
    print("\n" + "="*50 + f"\n--- BẢNG TỔNG HỢP KẾT QUẢ CHO x = {x_val} ---")
    
    # >>> SỬA LỖI Ở ĐÂY <<<
    # Áp dụng định dạng "{:g}" CHỈ cho cột chứa số
    display(summary_df.style.format("{:g}", subset=[column_name_value]))
    
    return summary_df

# --- 3. Thực thi thuật toán ---
try:
    ket_qua_dao_ham = tinh_P_va_cac_dao_ham(p_coeffs_descending, x_prime)
except NameError:
    print("LỖI: Biến 'p_coeffs_descending' chưa được định nghĩa.")
    print("Vui lòng chạy ô code đầu tiên (ô nhập file) trước.")


Sử dụng các hệ số P(x) (bậc giảm dần): [23. 34.  1. 56.]
Tính các đạo hàm tại x = 1.5

--- BƯỚC 1: TÍNH TOÁN CHO ĐẠO HÀM CẤP 0 ---


,a_3,a_2,a_1,a_0
Hệ số vào,23.0,34.0,1.00,56.000
Nhân với x=1.5,,34.5,102.75,155.625
Kết quả,23.0,68.5,103.75,211.625


-> Số dư R_1 = 211.625
-> Giá trị đạo hàm P^(0)(x') = R_1 * 0! = 211.625 * 1.0 = 211.625

--- BƯỚC 2: TÍNH TOÁN CHO ĐẠO HÀM CẤP 1 ---


,a_2,a_1,a_0
Hệ số vào,23.0,68.5,103.75
Nhân với x=1.5,,34.5,154.50
Kết quả,23.0,103.0,258.25


-> Số dư R_2 = 258.25
-> Giá trị đạo hàm P^(1)(x') = R_2 * 1! = 258.25 * 1.0 = 258.25

--- BƯỚC 3: TÍNH TOÁN CHO ĐẠO HÀM CẤP 2 ---


,a_1,a_0
Hệ số vào,23.0,103.0
Nhân với x=1.5,,34.5
Kết quả,23.0,137.5


-> Số dư R_3 = 137.5
-> Giá trị đạo hàm P^(2)(x') = R_3 * 2! = 137.5 * 2.0 = 275

--- BƯỚC 4: TÍNH TOÁN CHO ĐẠO HÀM CẤP 3 ---


,a_0
Hệ số vào,23.0
Nhân với x=1.5,
Kết quả,23.0


-> Số dư R_4 = 23
-> Giá trị đạo hàm P^(3)(x') = R_4 * 3! = 23 * 6.0 = 138

--- BẢNG TỔNG HỢP KẾT QUẢ CHO x = 1.5 ---


,Đạo hàm,Giá trị tại x = 1.5
0,P^(0)(x'),211.625
1,P^(1)(x'),258.25
2,P^(2)(x'),275
3,P^(3)(x'),138
